<a href="https://colab.research.google.com/github/gizdatalab/CPU/blob/main/classifiers/CONDITIONAL/CPU_CONDITIONAL_SentenceT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
! pip install datasets transformers sentencepiece huggingface_hub
! apt install git-lfs
! pip install sentence-transformers
! pip install evaluate
! pip install accelerate -U
! pip install codecarbon

In [2]:
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

[codecarbon INFO @ 18:35:08] [setup] RAM Tracking...
[codecarbon INFO @ 18:35:08] [setup] GPU Tracking...
[codecarbon INFO @ 18:35:08] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:35:08] [setup] CPU Tracking...
[codecarbon WARNING @ 18:35:08] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 18:35:10] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 18:35:10] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 18:35:10] >>> Tracker's metadata:
[codecarbon INFO @ 18:35:10]   Platform system: Linux-6.1.58+-x86_64-with-glibc2.35
[codecarbon INFO @ 18:35:10]   Python version: 3.10.12
[codecarbon INFO @ 18:35:10]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 18:35:10]   Available RAM : 12.675 GB
[codecarbon INFO @ 18:35:10]   CPU count: 2
[codecarbon INFO @ 18:35:10]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 18:35:10]   GPU c

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from datasets import load_dataset
import datasets
import numpy as np
import pandas as pd
import evaluate
import torch
import os
import sklearn.metrics as skm
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          PreTrainedModel, BertModel, BertForSequenceClassification,
                          RobertaForSequenceClassification,
                          TrainingArguments, Trainer, TrainerCallback)
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Manual Inputs


In [6]:
###### Define Path to data ############################
repo_id = "GIZ/policy_classification"
train_file="policy_classification_train.json"
test_file="policy_classification_test.json"


### Define labels ##################
label_names= ['UnconditionalLabel','ConditionalLabel']


#######  Define Model params ######################

# define the model checkpoint
model_checkpoint = "BAAI/bge-base-en-v1.5"

# define the problem type
problem_type="multi_label_classification"


######## Training Arguments ##################
output_dir = "CONDITIONAL-multilabel-bge"

# logging and evaluation
evaluation_strategy = "epoch"
logging_strategy="epoch"
logging_steps = 1
eval_steps = 1

# Important params
learning_rate=4.02e-5   #4.04e-5
batch_size = 16
num_train_epochs= 6
weight_decay=0.01
warmup_steps = 200
gradient_accumulation_steps = 1

# Automated

## Dataset

In [7]:
# Load Dataset from Hugging Face
data_files = {"train": train_file, "test": test_file}
dataset = load_dataset(repo_id, data_files=data_files)

# the dataset already contains train_test split flag we use
# it to create train-test split.

train_df = dataset['train'].to_pandas()
test_df =dataset['test'].to_pandas()

# Sort the labels and create plaecholders for id2label and label2id
label_names.sort()

num_labels = len(label_names)
id2label = {idx:label for idx, label in enumerate(label_names)}
label2id = {label:idx for idx, label in enumerate(label_names)}
print(id2label)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

{0: 'ConditionalLabel', 1: 'UnconditionalLabel'}


In [8]:
# # given the high imbalance in positive and negative examples, we downsample the
# # negative examples
data = train_df.copy()
data['check'] = data.apply(lambda x: True if (x['UnconditionalLabel'] == True)
                          or (x['ConditionalLabel'] == True) else False,axis=1)
temp = data[data.check==True].reset_index(drop=True)
print(len(temp))
temp2 = data[data.check ==False].reset_index(drop=True).sample(frac= 0.3, random_state=42)
print(len(temp2))
df = pd.concat([temp,temp2], ignore_index=True)


3056
2845


In [9]:
def prepare_multilabel_df(df, cols):
    """
    Takes the dataframe and list of columns which need to combined into
    One-hot-encoded labels.

    """
    df.dropna(subset = cols, inplace=True)
    df = df.reset_index(drop=True)
    df['labels'] = df.apply(lambda x: [int(x[label]) for label in cols],axis=1)
    # we have multilingual dataset ('en', 'fr','es'), in case of non-english the
    # translated Context is udner the column 'TranslatedContext'. We impute these
    # back into 'context' to work with only English dataset.
    df['Context'] = df.apply(lambda x: x['Context'] if x['Language'] == 'en'
                                    else x['TranslatedContext'][0],axis=1)
    df.rename(columns = {'Context':'text'}, inplace=True)

    df = df[['text','labels']+cols]
    return df

# creating multilabel dataset
train_df = prepare_multilabel_df(df, label_names)
test_df = prepare_multilabel_df(test_df, label_names)

print('\n','Classes Representation in Training Dataset:',len(train_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(train_df['labels'])]))

print('\n','Classes Representation in Test Dataset:',len(test_df))
for i,label in enumerate(label_names):
    print(i,".",label, ":", sum( [x[i] for x in list(test_df['labels'])]))


 Classes Representation in Training Dataset: 5901
0 . ConditionalLabel : 1986
1 . UnconditionalLabel : 1312

 Classes Representation in Test Dataset: 1190
0 . ConditionalLabel : 192
1 . UnconditionalLabel : 136


## Model

In [10]:
# problem_type, is not needed in tokenizer but keeping it for conformity
# https://huggingface.co/docs/transformers/main_classes/configuration?highlight=multi_label_classification#transformers.PretrainedConfig
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,problem_type=problem_type)

# Set the model config and assign it and device to model
config = AutoConfig.from_pretrained(model_checkpoint, num_labels=num_labels, id2label=id2label, label2id=label2id, problem_type=problem_type)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,config=config).to(device)

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

[codecarbon INFO @ 18:41:10] Energy consumed for RAM : 0.000475 kWh. RAM Power : 4.753043174743652 W
[codecarbon INFO @ 18:41:10] Energy consumed for all GPUs : 0.000990 kWh. Total GPU Power : 9.722282998175627 W
[codecarbon INFO @ 18:41:10] Energy consumed for all CPUs : 0.004250 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:41:10] 0.005715 kWh of electricity used since the beginning.


special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at BAAI/bge-base-en-v1.5 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# we need datasets format to work with
train_ds = datasets.Dataset.from_pandas(train_df[['text','labels']])
test_ds =  datasets.Dataset.from_pandas(test_df[['text','labels']])
train_ds = train_ds.shuffle(seed=7)
test_ds = test_ds.shuffle(seed=7)


# Need to tokenize the data using the tokenizer of the model
def tokenize_and_encode(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,
                        max_length=384)

cols = train_ds.column_names
cols.remove("labels")
print('Training data:',train_ds.num_rows)
print('Validation data:',test_ds.num_rows)

train_tokenized = train_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)
val_tokenized = test_ds.map(tokenize_and_encode, batched=True, remove_columns= cols)

# need this to avoid error due to type mismatch
# https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3
train_tokenized.set_format("torch")
train_tokenized = (train_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

val_tokenized.set_format("torch")
val_tokenized = (val_tokenized
          .map(lambda x : {"float_labels": x["labels"].to(torch.float)}, remove_columns=["labels"])
          .rename_column("float_labels", "labels"))

[codecarbon INFO @ 18:41:25] Energy consumed for RAM : 0.000495 kWh. RAM Power : 4.753043174743652 W
[codecarbon INFO @ 18:41:25] Energy consumed for all GPUs : 0.001083 kWh. Total GPU Power : 22.345398864844736 W
[codecarbon INFO @ 18:41:25] Energy consumed for all CPUs : 0.004427 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 18:41:25] 0.006005 kWh of electricity used since the beginning.


Training data: 5901
Validation data: 1190


Map:   0%|          | 0/5901 [00:00<?, ? examples/s]

Map:   0%|          | 0/1190 [00:00<?, ? examples/s]

Map:   0%|          | 0/5901 [00:00<?, ? examples/s]

Map:   0%|          | 0/1190 [00:00<?, ? examples/s]

## Metrics

In [12]:
multilabel_f1_metric = evaluate.load("f1", "multilabel")
multilabel_recall_metric = evaluate.load("recall","multilabel")
multilabel_precision_metric = evaluate.load("precision","multilabel")

def get_scores(y_pred, y_test, thresh=0.5, sigmoid=True):
    """
    Takes the Predictions and Truth Values, and return multilabel metrics
    aggregated on multiple strategies['micro', 'sample', 'weighted']

    """
    # we get raw function value, therefore we apply sigmoid to get to the
    # probabilities
    if problem_type == "multi_label_classification":
        y_pred = torch.from_numpy(y_pred)
        y_test = torch.from_numpy(y_test)
        if sigmoid:
          y_pred = y_pred.sigmoid()
          y_pred = (y_pred > thresh)

        return {
            "precision-micro": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="micro")["precision"],
            "precision-samples": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="samples")["precision"],
            "precision-weighted": multilabel_precision_metric.compute(predictions=y_pred, references=y_test, average="weighted")["precision"],
            "recall-micro": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="micro")["recall"],
            "recall-samples": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="samples")["recall"],
            "recall-weighted": multilabel_recall_metric.compute(predictions=y_pred, references=y_test, average="weighted")["recall"],
            "f1-micro": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="micro")["f1"],
            "f1-samples": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="samples")["f1"],
            "f1-weighted": multilabel_f1_metric.compute(predictions=y_pred, references=y_test, average="weighted")["f1"],
            }
    else:
        y_pred = torch.from_numpy(y_pred).argmax(dim=1)
        y_test = torch.from_numpy(y_test)

        report = skm.classification_report(y_test, y_pred, output_dict=True)
        df_report = pd.DataFrame(report).transpose()
        return {"Precision_macro": df_report.loc['macro avg']['precision'],
                "Precision_weighted": df_report.loc['weighted avg']['precision'],
                "Recall_macro": df_report.loc['macro avg']['recall'],
                "Recall_weighted": df_report.loc['weighted avg']['recall'],
                "F1-Score": df_report.loc['macro avg']['f1-score'],
                "Accuracy": skm.accuracy_score(y_test, y_pred)}


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return get_scores(predictions, labels)

## Training arguments

In [14]:
args = TrainingArguments(
    output_dir,
    logging_strategy=logging_strategy,
    evaluation_strategy = evaluation_strategy,
    save_strategy = "epoch",
    logging_steps = logging_steps,
    eval_steps= eval_steps,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    lr_scheduler_type = "cosine",
    gradient_accumulation_steps = gradient_accumulation_steps,
    warmup_steps = warmup_steps,
)

# Training

In [15]:
multi_trainer =  Trainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)

[codecarbon INFO @ 18:42:38] [setup] RAM Tracking...
[codecarbon INFO @ 18:42:38] [setup] GPU Tracking...
[codecarbon INFO @ 18:42:38] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 18:42:38] [setup] CPU Tracking...
[codecarbon WARNING @ 18:42:38] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 18:42:39] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 18:42:39] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 18:42:39] >>> Tracker's metadata:
[codecarbon INFO @ 18:42:39]   Platform system: Linux-6.1.58+-x86_64-with-glibc2.35
[codecarbon INFO @ 18:42:39]   Python version: 3.10.12
[codecarbon INFO @ 18:42:39]   CodeCarbon version: 2.3.4
[codecarbon INFO @ 18:42:39]   Available RAM : 12.675 GB
[codecarbon INFO @ 18:42:39]   CPU count: 2
[codecarbon INFO @ 18:42:39]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 18:42:39]   GPU c

In [ ]:
multi_trainer.train()

In [17]:
tracker.stop()

[codecarbon INFO @ 19:24:55] Energy consumed for RAM : 0.003936 kWh. RAM Power : 4.753043174743652 W
[codecarbon INFO @ 19:24:55] Energy consumed for all GPUs : 0.050035 kWh. Total GPU Power : 51.752134827402344 W
[codecarbon INFO @ 19:24:55] Energy consumed for all CPUs : 0.035225 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:24:55] 0.089197 kWh of electricity used since the beginning.


0.031149905926205902

In [19]:
multi_trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

events.out.tfevents.1709664168.0e7496e21892.163.0:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/ppsingh/CONDITIONAL-multilabel-bge/commit/e1798933f8bcb782b24540e1da92e230dac45576', commit_message='End of training', commit_description='', oid='e1798933f8bcb782b24540e1da92e230dac45576', pr_url=None, pr_revision=None, pr_num=None)

In [20]:
log_results = multi_trainer.state.log_history.copy()
logs = []
while len(log_results)>1:

    a = log_results[0]
    b = log_results[1]
    logs.append({**a,**b})
    log_results.pop(0)
    log_results.pop(0)

results_df = pd.DataFrame(logs)
results_df

,loss,grad_norm,learning_rate,epoch,step,eval_loss,eval_precision-micro,eval_precision-samples,eval_precision-weighted,eval_recall-micro,eval_recall-samples,eval_recall-weighted,eval_f1-micro,eval_f1-samples,eval_f1-weighted,eval_runtime,eval_samples_per_second,eval_steps_per_second
0,0.5361,2.865768,0.000040,1.0,369,0.440495,0.340461,0.165546,0.410248,0.631098,0.162185,0.631098,0.442308,0.162185,0.450281,28.5693,41.653,2.625
1,0.3692,4.371844,0.000034,2.0,738,0.343733,0.463115,0.179412,0.492856,0.689024,0.176050,0.689024,0.553922,0.176190,0.560411,28.5185,41.727,2.630
2,0.1820,7.660297,0.000023,3.0,1107,0.391488,0.470250,0.185714,0.487098,0.746951,0.189076,0.746951,0.577150,0.185434,0.579955,28.3470,41.980,2.646
3,0.0757,3.075250,0.000012,4.0,1476,0.471314,0.495984,0.188235,0.498607,0.753049,0.190756,0.753049,0.598063,0.187675,0.598705,28.4009,41.900,2.641
4,0.0298,8.608804,0.000003,5.0,1845,0.497116,0.516129,0.184034,0.518429,0.731707,0.185714,0.731707,0.605296,0.182913,0.605844,28.5182,41.728,2.630
5,0.0152,0.145633,0.000000,6.0,2214,0.529515,0.513800,0.186555,0.516876,0.737805,0.187395,0.737805,0.605757,0.185154,0.606467,28.7040,41.458,2.613


# Predict and Evaluate

## Trainer

In [21]:
predictions= multi_trainer.predict(val_tokenized)
pred,labels,_ = predictions
y_pred = torch.from_numpy(pred)
y_true = torch.from_numpy(labels)
y_prob = y_pred.sigmoid()
thresh = 0.5
y_pred = (y_prob>=thresh).bool()
y_true = y_true.bool()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [22]:
cm = skm.multilabel_confusion_matrix(y_true, y_pred)
for i,label in enumerate(label_names):
    print(label)
    print(cm[i],'\n')

ConditionalLabel
[[846 152]
 [ 46 146]] 

UnconditionalLabel
[[977  77]
 [ 40  96]] 



In [23]:
report = skm.classification_report(y_true, y_pred, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report = df_report.reset_index()
mapping = {str(i):j for i,j in enumerate(label_names)}
df_report['index'] = df_report['index'].map(mapping).fillna(df_report['index'])
df_report.rename(columns = {'index':'label'}, inplace=True)
df_report

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,label,precision,recall,f1-score,support
0,ConditionalLabel,0.489933,0.760417,0.595918,192.0
1,UnconditionalLabel,0.554913,0.705882,0.621359,136.0
2,micro avg,0.513800,0.737805,0.605757,328.0
3,macro avg,0.522423,0.733150,0.608639,328.0
4,weighted avg,0.516876,0.737805,0.606467,328.0
5,samples avg,0.186555,0.187395,0.185154,328.0
